# SCAL3 demo

This demo uses the [Rust prototype](src/README.md) via its foreign function interface (FFI).

## Installation

Ensure you have compiled the prototype with `cargo build --release`.

In [41]:
%pip install cbor2 fastecdsa

from ctypes import cdll, memmove, string_at
from cbor2 import dumps, loads
from fastecdsa import keys, curve, ecdsa
from fastecdsa.encoding.sec1 import SEC1Encoder
from fastecdsa.encoding.util import int_to_bytes
import secrets
import hmac
import hashlib

Note: you may need to restart the kernel to use updated packages.


In [42]:
lib = cdll.LoadLibrary("target/release/libscal3.so")

We let Rust allocate a request/response buffer with sufficient bytes.

In [43]:
size = lib.scal3_buffer_size()
buf = lib.scal3_buffer_allocate()

## Helper functions

In [44]:
def write_request(request):
    data = dumps(request)
    memmove(buf, data, len(data))
def read_response():
    return loads(string_at(buf, size))
def ecdh(sk, pk):
    pk = SEC1Encoder().decode_public_key(pk, curve.P256)
    return SEC1Encoder().encode_public_key(sk * pk, compressed=False)[1:33]

## Setup

Create a single key pair and a PRF key for the provider. In production, these would be HSM-backed.

In [45]:

sk_provider, pk_provider = keys.gen_keypair(curve.P256)
pk_provider = SEC1Encoder.encode_public_key(pk_provider)
k_provider = secrets.token_bytes(32)

## Enrolment

The subscriber creates a possession key pair and a PRF key. In production, these would be backed by mobile secure hardware.

In [46]:

sk_device, pk_device = keys.gen_keypair(curve.P256)
pk_device = SEC1Encoder.encode_public_key(pk_device)
k_subscriber = secrets.token_bytes(32)

The subscriber creates a registration request. In production, this request would additionally be signed using an (attested) possession public key.

In [47]:
write_request({
    'mask': hmac.digest(k_subscriber, b"123456", hashlib.sha256),
    'randomness': secrets.token_bytes(32),
    'provider': pk_provider,
})
status = lib.scal3_subscriber_register(buf)
response = read_response()
subscriber = response['subscriber']
verifier = response['verifier']

Upon processing the registration request, the provider verifies if it is formed in a valid way.

In [48]:
credential = {
    'verifier': verifier,
    'device': pk_device,
}
provider_state = credential | {
    'provider': pk_provider,
    'verifierSecret': ecdh(sk_provider, subscriber),
}
write_request(provider_state)
status = lib.scal3_provider_accept(buf)
response = read_response()

## Authentication

The provider creates a challenge over some data. The combination is shared with a subscriber.

In [49]:
challenge_data = b"ts=1743930934&nonce=000001"
write_request({
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256)
})
status = lib.scal3_provider_challenge(buf)
response = read_response()
challenge = response['challenge']

The subscriber enters their PIN bound to some client data and uses the possession factor to create a signature.

In [50]:
client_data = b'{"operation":"log-in","session":"68c9eeeddfa5fb50"}'
client_data_hash = hashlib.sha256(client_data).digest()
write_request(credential | {
    'mask': hmac.digest(k_subscriber, b"123456", hashlib.sha256),
    'randomness': secrets.token_bytes(32),
    'provider': pk_provider,
    'subscriber': subscriber,
    'challenge': challenge,
    'hash': client_data_hash,
})
authentication = lib.scal3_subscriber_authenticate(buf)
response = read_response()
digest = response['digest']

r, s = ecdsa.sign(digest, sk_device, prehashed=True)
write_request({ 'proof': int_to_bytes(r, 32) + int_to_bytes(s, 32) })
status = lib.scal3_subscriber_pass(authentication, buf)
response = read_response()
sender = response['sender']
pass_attempt = response['pass']

Upon receiving back the challenge and the authentication attempt data, the provider tries to prove authentication.

In [51]:
write_request(provider_state | {
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256),
    'hash': client_data_hash,
    'passSecret': ecdh(sk_provider, sender),
    'pass': pass_attempt
})
status = lib.scal3_provider_prove(buf)
transcript = read_response()

## Auditing

Anyone can verify proven authentication.

In [55]:
evidence = credential | transcript | { 'hash': client_data_hash }
evidence

{'verifier': b'\xdb)J%\xcb;W\x8b<3R`\x81(\xfe)Lb\x89q_\xe3<\xa5\xbe\xd18\xfb\x966\xd6\xad\x02N\xf01\x92\xba\x93h\x95m\x88g\x0fHM1j\xbc5p\x1d\xc5\xf6\xecI\\!\xa9\xa1\xf8\xdb0r\x03gf\xf5\xc8\n\x05dn\x19\x8e\x87\xcf\xdb\x00\xe7= \xb2\x00\xb4\x87\x9d\xed\x04\x1ds:\n}\x14\xd5\xea,\x11\xbd,\xa47Z\xb32h\x13DKs\x8cs1~K\x16\n{\x85\n\x02s\x8c&2\xc8^\x17g(\xe1\xa2\x8cV\xfci\x83\x0f\x96X\xde\\ \x0eRD\x8c\x0f\xdaa\xae\x0b]\xc2\x9c\xce\x95,\xb5g:XP/\x1f\xbe\x01\xdf_\x03s\xe4\r\xd6\xb9\x8e\x1a\x1a8\x05\xd1\xd7\xa7g\x1e\x9d{\x8b\xbd\x1e\xc9WY\xcd\x8a`\xa2@|\xab\x84/\x8c%\xaa\xb4\xa1\xabBB\xb8>\xb3~\x14\x0f\xbd\x08\x81\x16=JN\x03\x19\n\xd0Y\xc8\x12G\xf7ol\x91\xa5Q\xdc\xdb\xac\x16Ps\x9c\x7f.(\xc8',
 'device': b'\x02\xd6\x94\xcb\xaf1\x93\xb7\xf7EjG\xbbv;\xa5m\x81\x9a\xc2\xcep\x81\x19#\xccr\xcf\xe6\x1b~\xd0\\',
 'authenticator': b"\x86?\x8b'v\x80\xafe\xd08\xab[7\x86\x91\xdcL$\xac\xf4\xae\x12\xe4\xae\xd1K\xa5c\xa0\x19\xb8\xefRm\xbf\xf8Noc<\xd4\x02V~|\x19\xa3\xe9",
 'proof': b'\x97j\x11&=\x19\xd2\x86\xf2\xb

In [ ]:
write_request(evidence)
status = lib.scal3_verify(buf)
response = read_response()

assert status == 0
assert response['result'] == 'verified'

## Teardown

Ensure to free the buffer after using it.

In [53]:
lib.scal3_buffer_free(buf)

0